# STGCN：路网交通预测
## 论文资料
- [Spatio-Temporal Graph Convolutional Networks: A Deep Learning Framework for Traffic](https://arxiv.org/abs/1709.04875v4)
- [原文代码](https://github.com/VeritasYin/STGCN_IJCAI-18)
## 参考资料
- [论文阅读笔记](https://davidham3.github.io/blog/2018/05/10/spatio-temporal-graph-convolutional-networks-a-deep-learning-framework-for-traffic/)
- [mxnet复现代码](https://github.com/Davidham3/STGCN)
- [STGCN论文详解](https://zhuanlan.zhihu.com/p/78259670)
## 简介
实时精确的交通预测对城市交通管控和引导很重要。由于交通流的强非线性以及复杂性，传统方法并不能满足中长期预测的要求，而且传统方法经常忽略对时空数据的依赖。在这篇论文中，作者提出了一个新的深度学习框架，时空图卷积(Spatio-Temporal Graph Convolutional Networks)，来解决交通领域的时间序列预测问题。

在交通研究中，交通流的基本变量，也就是速度、流量和密度，通常作为监控当前交通状态以及未来预测的指示指标。根据预测的长度，交通预测大体分为两个尺度：短期(5~30min)，中和长期预测(超过30min)。大多数流行的统计方法(比如，线性回归)可以在短期预测上表现的很好。然而，由于交通流的不确定性和复杂性，这些方法在相对长期的预测上不是那么的有效。

交通预测是一个典型的时间序列预测问题，也就是预测在给定前M个观测样本接下来H个时间戳后最可能的交通流指标（比如速度或交通流）：

$\hat{v}_{t+1}, ..., \hat{v}_{t+H} = \mathop{\arg\min}_{v_{t+1},...,v_{t+H}}logP(v_{t+1},...,v_{t+H}\vert v_{t-M+1},...v_t)$

这里$v_t \in \mathbb{R}^n$是$n$个路段在时间戳$t$观察到的一个向量，每个元素记录了一条路段的历史观测数据。

作者在一个图上定义了一个交通网络，并专注于结构化的交通时间序列。观测到的样本v_t间不是相互独立的，而是在图中两两相互连接的。因此，数据点$v_t$可以被视为定义在权重为$w_{ij}$，如下图展示的无向图（或有向图）$\mathcal{G}$上的一个信号。在第$t$个时间戳，在图$\mathcal{G_t}=(\mathcal{V_t}, \mathcal{\varepsilon}, W), \mathcal{V_t}$是当顶点的有限集，对应在交通网络中n个监测站；$\epsilon$是边集，表示观测站之间的连通性；$W \in \mathbb{R^{n \times n}}$表示$\mathcal{G_t}$的邻接矩阵。

![file](https://pic1.zhimg.com/v2-5d5efa123e08b8e7cca00b327843aeb7_1440w.jpg)

> **关于图学习的概念，可以参考[PGL：Paddle带你走进图学习](https://aistudio.baidu.com/aistudio/projectdetail/413386)系列课程。**

> 关于交通网络的时空相关性，也可以参考其它论文给出的更详细示意，如：
>
> [论文：面向交通流量预测的多组件时空图卷积网络](http://www.jos.org.cn/html/2019/3/5697.htm)
>
> 交通流量预测是典型的时空数据预测问题, 不同类别的交通数据内嵌于连续空间, 并且随时间动态变化, 因此, 有效提取时空相关性对解决这类问题至关重要.下图所示为流量数据(也可以是车速、车道占用率等其他交通数据)的时空相关性示意图, 时间维包含3个时间片, 空间维的6个节点(A~F)表示公路的网状结构。在空间维上, 节点的交通状况之间会相互影响(绿色虚线); 时间维上, 某节点历史不同时刻流量会对该节点未来不同时刻流量产生影响(蓝色虚线); 同时, 节点历史不同时刻的流量值也会对其关联节点未来不同时刻的流量产生影响(红色虚线)。可见, 交通流量在时空维度都存在很强的相关性。
>
> ![file](http://www.jos.org.cn/html/2019/3/PIC/rjxb-30-3-759-1.jpg)

## 网络结构
STGCN有多个时空卷积块组成，每一个都是像一个“三明治”结构的组成，有两个门序列卷积层和一个空间图卷积层在中间。

![file](https://pic3.zhimg.com/80/v2-030389b5592ad95cc19e3546ae70510e_720w.jpg)

STGCN的架构有两个时空卷积块和一个全连接的在末尾的输出层组成。每个ST-Conv块包含了两个时间门卷积层，中间有一个空间图卷积层。每个块中都使用了残差连接和bottleneck策略。输入$v_{t-M+1},…v_t$被ST-Conv块均匀（uniformly）处理，来获取时空依赖关系。全部特征由一个输出层来整合，生成最后的预测$\hat{v}$。

## PGL实现
### 安装工具库

In [ ]:
!git clone https://gitee.com/paddlepaddle/PGL.git

Cloning into 'PGL'...
remote: Enumerating objects: 1824, done.
remote: Counting objects: 100% (1824/1824), done.
remote: Compressing objects: 100% (1224/1224), done.
remote: Total 1824 (delta 977), reused 1061 (delta 528), pack-reused 0
Receiving objects: 100% (1824/1824), 16.62 MiB | 3.40 MiB/s, done.
Resolving deltas: 100% (977/977), done.
Checking connectivity... done.


In [ ]:
!pip install pgl

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
     |████████████████████████████████| 7.9MB 48kB/s eta 0:00:013
     |████████████████████████████████| 71kB 32.9MB/s eta 0:00:01


### 数据集准备
PeMSD7是Caltrans Performance Measurement System(PeMS)通过超过39000个监测站实时获取的数据，这些监测站分布在加州高速公路系统主要的都市部分。数据是30秒的数据样本聚合成5分钟一次的数据。作者在加州的District 7随机选取了一个小的和一个大的范围作为数据源，分别有228和1026个监测站，分别命名为PeMSD7(S)和PeMSD7(L)。PeMSD7数据集的时间范围是2012年五月和六月的周末，选取了第一个月的车速速度记录作为训练集，剩下的分别做验证和测试。

![file](https://ai-studio-static-online.cdn.bcebos.com/0ad23394ba10462993573f0c0e26014c80cdec467ded4176a4683e895bb54ac9)


### 数据预处理
路网中的每个顶点（传感器）每天就有288个数据点。数据清理后使用了线性插值的方法来填补缺失值。通过核对相关性，每条路的方向和OD(origin-destination)点，环路系统可以被数值化成一个有向图。

在PeMSD7，路网的邻接矩阵通过交通网络中的监测站的距离来计算。带权邻接矩阵W通过以下公式计算：

$w_{ij} = \begin{cases}
\exp{(-\frac{d^2_{ij}}{\sigma^2})}&,i \neq j \ \rm and \exp{(-\frac{d^2_{ij}}{\sigma^2}) \geq \epsilon} \\
0&, \rm otherwise
\end{cases}$

其中$w_{ij}$是边的权重，通过$d_{ij}$得到，也就是$i$和$j$之间的距离。$\sigma^2$和$\epsilon$是来控制矩阵W的分布和稀疏性的阈值，文中用了10和0.5。$W$的可视化在上图的右侧。

[作者代码](https://github.com/VeritasYin/STGCN_IJCAI-18)中给出了PeMS-M数据集的压缩包，本文已将数据集解压放在PeMS-M目录下。
```
PeMS-M/
    -- W_228.csv
    -- V_228.csv
```
其中，`V_228.csv`是传感器记录的车速信息，`W_228.csv`是已经处理好的邻接矩阵。

### 开始训练
> 由于没能跑通PGL给出的示例代码，这里对`PGL/examples/stgcn`目录下的`data_loader/data_utils.py`和`main.py`稍作修改，参考[mxnet复现代码](https://github.com/Davidham3/STGCN)重写了`data_loader/data_utils.py`里的部分函数。

In [1]:
%cd PGL/examples/stgcn/

/home/aistudio/PGL/examples/stgcn


In [ ]:
%run main.py --use_cuda --input_file /home/aistudio/PeMS-M/V_228.csv --adj_mat_file /home/aistudio/PeMS-M/W_228.csv

[INFO] 2020-06-27 10:25:01,538 [     main.py:  174]:	Namespace(Ks=3, Kt=3, adj_mat_file='/home/aistudio/PeMS-M/W_228.csv', batch_size=10, blocks=[[1, 32, 64], [64, 32, 128]], epochs=5, inf_mode='sep', input_file='/home/aistudio/PeMS-M/V_228.csv', keep_prob=1.0, lr=0.001, n_his=9, n_pred=3, n_route=228, opt='ADAM', output_path='./outputs/', save=1, use_cuda=True)
[INFO] 2020-06-27 10:25:04,711 [     main.py:   41]:	{'mean': 59.49499979949002, 'std': 13.170890048189376}
[INFO] 2020-06-27 10:25:04,712 [     main.py:   42]:	7583
[INFO] 2020-06-27 10:25:08,762 [     main.py:  109]:	epoch 1 | step 0 | lr 0.001000 | loss 37735.941406
[INFO] 2020-06-27 10:25:13,379 [     main.py:  109]:	epoch 1 | step 5 | lr 0.001000 | loss 18736.484375
[INFO] 2020-06-27 10:25:17,919 [     main.py:  109]:	epoch 1 | step 10 | lr 0.001000 | loss 18630.222656
[INFO] 2020-06-27 10:25:22,470 [     main.py:  109]:	epoch 1 | step 15 | lr 0.001000 | loss 13020.451172
[INFO] 2020-06-27 10:25:27,010 [     main.py:  109]

Time Step 3: MAPE 13.896%, 12.535%; MAE  4.914, 4.759; RMSE  7.973,  7.636.
Time Step 1: MAPE 11.567%; MAE  4.552; RMSE  7.401.
Time Step 2: MAPE 11.927%; MAE  4.595; RMSE  7.437.
Time Step 3: MAPE 12.535%; MAE  4.759; RMSE  7.636.


[INFO] 2020-06-27 10:44:26,944 [     main.py:  109]:	epoch 2 | step 0 | lr 0.001000 | loss 5626.972656
[INFO] 2020-06-27 10:44:31,626 [     main.py:  109]:	epoch 2 | step 5 | lr 0.001000 | loss 5328.616699
[INFO] 2020-06-27 10:44:36,273 [     main.py:  109]:	epoch 2 | step 10 | lr 0.001000 | loss 4990.459961
[INFO] 2020-06-27 10:44:40,917 [     main.py:  109]:	epoch 2 | step 15 | lr 0.001000 | loss 4208.468750
[INFO] 2020-06-27 10:44:45,592 [     main.py:  109]:	epoch 2 | step 20 | lr 0.001000 | loss 5790.794922
[INFO] 2020-06-27 10:44:50,268 [     main.py:  109]:	epoch 2 | step 25 | lr 0.001000 | loss 4302.013672
[INFO] 2020-06-27 10:44:54,961 [     main.py:  109]:	epoch 2 | step 30 | lr 0.001000 | loss 6110.498535
[INFO] 2020-06-27 10:44:59,642 [     main.py:  109]:	epoch 2 | step 35 | lr 0.001000 | loss 6567.183594
[INFO] 2020-06-27 10:45:04,327 [     main.py:  109]:	epoch 2 | step 40 | lr 0.001000 | loss 3542.793457
[INFO] 2020-06-27 10:45:08,983 [     main.py:  109]:	epoch 2 | ste

# STGCN：高致病性传染病传播趋势预测基线系统学习
- [比赛链接](https://aistudio.baidu.com/aistudio/competition/detail/36)
- [基于飞桨PGL的基线系统](https://aistudio.baidu.com/aistudio/projectdetail/464528)

在该场景中，官方baseline使用STGCN进行传播趋势预测，并没有改动图结构和STGCN网络，重点更多在数据预处理上。与交通流量的自回归不同的是，增加了label也就是感染人数。

本文基于官方baseline略作修改，并增加了数据预处理阶段的注释，目录结构如下：
```
work/
  -- dataset/
  -- dataloader.py 
  -- Dataset.py
  -- graph.py
  -- model.py
  -- main.py
```

## 解压数据集

In [ ]:
%cd /home/aistudio/work

/home/aistudio/work


In [ ]:
# unzip dataset
%mkdir ./dataset/
!unzip /home/aistudio/data/data33637/train_data.zip -d ./dataset/

Archive:  /home/aistudio/data/data33637/train_data.zip
   creating: ./dataset/train_data/
   creating: ./dataset/train_data/city_C/
  inflating: ./dataset/train_data/city_C/migration.csv  
  inflating: ./dataset/train_data/city_C/density.csv  
  inflating: ./dataset/train_data/city_C/transfer.csv  
  inflating: ./dataset/train_data/city_C/weather.csv  
  inflating: ./dataset/train_data/city_C/grid_attr.csv  
  inflating: ./dataset/train_data/city_C/infection.csv  
   creating: ./dataset/train_data/city_D/
  inflating: ./dataset/train_data/city_D/density.csv  
  inflating: ./dataset/train_data/city_D/migration.csv  
  inflating: ./dataset/train_data/city_D/transfer.csv  
  inflating: ./dataset/train_data/city_D/weather.csv  
  inflating: ./dataset/train_data/city_D/grid_attr.csv  
  inflating: ./dataset/train_data/city_D/infection.csv  
   creating: ./dataset/train_data/city_E/
  inflating: ./dataset/train_data/city_E/density.csv  
  inflating: ./dataset/train_data/city_E/migration.csv 

## 引入工具库

In [ ]:
"""data processing
"""
import os
import sys
import argparse
import numpy as np
import pandas as pd
from collections import defaultdict, OrderedDict
import pdb
import gc

## 获取指定城市每个地点经纬度以及归属区域
根据`grid_attr.csv`给出的城市地点经纬度详细信息，将其转为`<key,value>`的形式。

其实这种做法有个前提，那就是每个城市的`grid_attr.csv`包括了迁移数据中起始点和到达点的全部经纬度。假设在真实场景下，可以考虑用电子围栏。

In [ ]:
def get_grid_dict(city_path, city_name):
    d = {}
    with open(os.path.join(city_path, 'grid_attr.csv'), 'r') as f:
        for line in f:
            items = line.strip().split(',')
            axis = ",".join(items[0:2])
            ID = items[2]
            d[axis] = "_".join([city_name, ID])
    # print(d)
    # d = {'x,y': ID}
    return d

## 计算市内区域迁移指数
反映人群迁移情况的`transfer.csv`里起始点和到达点都是经纬度，这里需要将其换算为所属的城市区域ID，通过查找`grid_dict`里面的`<key,value>`列表实现。

In [ ]:
def coord2ID(data_path, city_name, output_path):
    city_path = os.path.join(data_path, "city_%s" % city_name)
    grid_dict = get_grid_dict(city_path, city_name)
    # grid_dict = {'x,y': ID}
    trans_filename = os.path.join(city_path, "transfer.csv")
    output_file = os.path.join(output_path, "%s_transfer.csv" % (city_name))
    with open(trans_filename, 'r') as f, open(output_file, 'w') as writer:
        for line in f:
            items = line.strip().split(',')
            start_axis = ",".join(items[1:3])
            end_axis = ",".join(items[3:5])
            index = items[5]
            try:
                start_ID = grid_dict[start_axis]
                end_ID = grid_dict[end_axis] 
            except KeyError: # remove no ID axis
                continue

            writer.write("%s,%s,%s,%s\n" % (items[0], start_ID, end_ID, index))

In [ ]:
coord2ID('./dataset/train_data', 'A', './dataset')

In [ ]:
# 查看得到的每小时区域人群迁移指数
pd.read_csv('dataset/A_transfer.csv', header=None, names=['hour', 's_region', 'e_region', 'index']).head()

,hour,s_region,e_region,index
0,0,A_0,A_0,0.1
1,0,A_0,A_0,0.2
2,0,A_0,A_0,0.1
3,0,A_0,A_0,0.1
4,0,A_0,A_0,0.1


In [ ]:
# 求和计算每日市内区域人群迁移指数
def calc_index_in_one_day(data_path, city_name):
    trans_filename = os.path.join(data_path, "%s_transfer.csv" % (city_name))
    transfer = pd.read_csv(trans_filename, 
            header=None,
            names=['hour', 's_region', 'e_region', 'index'])
        
    df = transfer.groupby(['s_region', 'e_region'])['index'].sum().reset_index()
    df = df[['s_region', 'e_region', 'index']]
    #  df = df.T
    #  df_list.append(df)
    return df

In [ ]:
calc_index_in_one_day('./dataset', 'A')

,s_region,e_region,index
0,A_0,A_0,187.5
1,A_0,A_1,0.3
2,A_0,A_10,0.8
3,A_0,A_100,0.2
4,A_0,A_108,0.4
5,A_0,A_11,0.7
6,A_0,A_110,0.1
7,A_0,A_111,0.1
8,A_0,A_112,0.3
9,A_0,A_116,0.1


## 计算城市迁移指数
每个城市的`migration.csv`记录了从该城市出发、到达该城市的人流量指数，可以通过统计计算每日到达指定城市的人流量。

In [ ]:
def process_city_migration(data_path, city_name):
    filename = os.path.join(data_path, "city_%s" % city_name, "migration.csv")
    migration = pd.read_csv(filename, 
                            sep=',', 
                            header=None,
                            names=['date', 's_city', 'e_city', city_name])

    # only use moving in "city" data, ignore moving out data
    df = migration[migration.e_city == city_name]
    df = df[["date", city_name]]

    # calculate total move in data of "city"
    df = df.groupby('date')[city_name].sum().reset_index()
    return df

In [ ]:
# 计算每日到达A地的人流量
process_city_migration('./dataset/train_data', 'A')

,date,A
0,21200501,0.811620
1,21200502,0.742641
2,21200503,0.964937
3,21200504,0.771767
4,21200505,0.727024
5,21200506,1.101211
6,21200507,0.750903
7,21200508,1.004562
8,21200509,0.887760
9,21200510,0.890514


In [ ]:
def migration_process(data_path, city_list, output_path):
    for city_name in city_list:
        coord2ID(data_path, city_name, output_path)
        transfer = calc_index_in_one_day(output_path, city_name)
        migration = process_city_migration(data_path, city_name)

        df_list = []
        for i in range(len(migration)):
            df = transfer.copy()
            date = migration.date[i]
            index = migration[city_name][i]
            # 这里通过将到达城市的人流量指数与市内区域人流量指数相乘，得到区域人流量指数
            df['index'] = df['index'] * index
            df['date'] = date
            df = df[['date', 's_region', 'e_region', 'index']]
            # 按日新增区域人流量统计数据
            df_list.append(df)

        df = pd.concat(df_list, axis=0)
        # 得到每个城市最终迁移人流量指数
        df.to_csv(os.path.join(output_path, '%s_migration.csv' % city_name), 
                header=None,
                index=None,
                float_format = '%.4f')

In [ ]:
migration_process('./dataset/train_data', ["A", "B", "C", "D", "E"], './dataset')

In [ ]:
pd.read_csv('dataset/A_migration.csv', header=None).head()

,0,1,2,3
0,21200501,A_0,A_0,152.1787
1,21200501,A_0,A_1,0.2435
2,21200501,A_0,A_10,0.6493
3,21200501,A_0,A_100,0.1623
4,21200501,A_0,A_108,0.3246


## 计算邻接矩阵

In [ ]:
def adj_matrix_process(data_path, city_list, region_nums, output_path):
    total_region_num = np.sum(region_nums)
    adj_matrix = np.zeros((total_region_num, total_region_num))

    offset = 0
    for i, city in enumerate(city_list):
        filename = os.path.join(output_path, "%s_migration.csv" % city)
        migration = pd.read_csv(filename, 
                                sep=',', 
                                header=None,
                                names=['date', 's_region', 'e_region', 'index'])
        # 生成每个城市的初始邻接矩阵，比如A城市是有118个区域，shape就是(118, 118)
        matrix = np.zeros((region_nums[i], region_nums[i]))
        # pdb.set_trace()
        # 对区域编码进行排序，官方baseline的写法会把10,100排到2,3,4等前面
        # order = sorted(range(region_nums[i]), key=lambda x:str(x))
        order = sorted(list(range(region_nums[i])))
        for j, idx in enumerate(order):
            # 拼接目标区域的标准名称：城市名称+区域ID
            target_region = "%s_%d" % (city, idx)
            # only use moving in "city" data, ignore moving out data
            # 只用到迁入城市的人流量，不考虑迁出的问题
            df = migration[migration['e_region'] == target_region]

            # 计算得到每个区域迁入的平均人流量
            df = df.groupby('s_region')['index'].mean().reset_index()
            #  res = df['index'].values.reshape(-1)
            for k, o in enumerate(order):
                s_region_id = "%s_%d" % (city, o)
                try:
                    # 取出来自指定出发地的平均人流量数据
                    value = df[df['s_region'] == s_region_id]['index'].values[0]
                except:
                    value = 0.0
                if s_region_id == target_region:
                    value = 0.0
                # 给邻接矩阵该位置的元素赋值
                matrix[j, k] = value

        # merge two adj_matrix
        # 把不同城市的邻接矩阵拼起来，形成最终的大的邻接矩阵
        adj_matrix[offset:(offset + region_nums[i]), offset:(offset + region_nums[i])] = matrix
        offset += region_nums[i]
    # 这里调整了一下，保存为csv格式
    file_to_save = os.path.join(output_path, 'adj_matrix.csv')
    print("saving result to %s" % file_to_save)
    # np.save(file_to_save, adj_matrix)
    np.savetxt(file_to_save, adj_matrix, delimiter=',')

In [ ]:
adj_matrix_process('./dataset/train_data', ["A", "B", "C", "D", "E"], [118, 30, 135, 75, 34], './dataset')

saving result to ./dataset/adj_matrix.csv


In [ ]:
pd.read_csv('dataset/adj_matrix.csv',header=None).head()

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,0.000000,0.076853,0.307436,0.000000,0.384284,0.000000,0.230571,0.230571,0.000000,0.230571,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.230571,0.000000,0.153711,0.230571,0.076853,0.999138,0.076853,0.153711,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.153711,0.307436,0.000000,3.996533,13.219304,0.537987,0.845416,1.767704,1.460271,4.611380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,3.535398,0.000000,8.377353,0.076853,6.993933,1.844553,1.152851,5.379951,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.230571,0.000000,14.372151,8.069929,0.000000,0.922280,2.613129,7.685642,4.841956,18.906689,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 感染人数处理

In [ ]:
pd.read_csv('dataset/train_data/city_A/infection.csv', header=None, names=["city", "region", "date", "infect"]).head()

,city,region,date,infect
0,A,0,21200501,0
1,A,0,21200502,0
2,A,0,21200503,0
3,A,0,21200504,0
4,A,0,21200505,0


In [ ]:
def infection_process(data_path, city_list, region_nums, output_path):
    res = []
    region_name_list = []
    for i, city in enumerate(city_list):
        filename = os.path.join(data_path, "city_%s" % city, "infection.csv")
        migration = pd.read_csv(filename, 
                                sep=',', 
                                header=None,
                                names=["city", "region", "date", "infect"])

        # order = sorted(range(region_nums[i]), key=lambda x:str(x))
        order = sorted(list(range(region_nums[i])))
        for j, idx in enumerate(order):
            target_region = idx #str(idx)
            # pdb.set_trace()
            # 区域每天感染人数
            df = migration[migration['region'] == target_region].reset_index(drop=True)
            if i == 0 and j == 0:
                # 第一个区域要把日期给进去
                df = df[['date', 'infect']]
            else:
                df = df[['infect']]

            df = df.rename(columns={'infect': '%s_%d' % (city, idx)})
            region_name_list.append("%s_%d" % (city, idx))

            res.append(df)
    df = pd.concat(res, axis=1)
    # 最终形成城市+区域ID形式的感染人数大宽表
    file_to_save = os.path.join(output_path, "infection.csv")
    print("saving result to %s" % file_to_save)
    # format: [date, A, B, C, D, E]
    df.to_csv(file_to_save, index=False)

    region_name_file = os.path.join(output_path, "region_names.txt")
    with open(region_name_file, 'w') as f:
        names = ' '.join(region_name_list)
        f.write(names + '\n')

In [ ]:
infection_process('./dataset/train_data', ["A", "B", "C", "D", "E"], [118, 30, 135, 75, 34], './dataset')

saving result to ./dataset/infection.csv


In [ ]:
pd.read_csv('dataset/infection.csv').head()

,date,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,...,E_24,E_25,E_26,E_27,E_28,E_29,E_30,E_31,E_32,E_33
0,21200501,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21200502,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,21200503,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,21200504,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21200505,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 迁移人流量处理

In [ ]:
def region_migration_process(data_path, city_list, region_nums, output_path):
    res = []
    # 这里和感染人数处理类似
    for i, city in enumerate(city_list):
        filename = os.path.join(output_path, "%s_migration.csv" % city)
        migration = pd.read_csv(filename, 
                                sep=',', 
                                header=None,
                                names=['date', 's_region', 'e_region', 'index'])

        # order = sorted(range(region_nums[i]), key=lambda x:str(x))
        order = sorted(list(range(region_nums[i])))
        for j, idx in enumerate(order):
            target_region = "%s_%d" % (city, idx)
            df = migration[migration['e_region'] == target_region]

            df = df.groupby('date')['index'].sum().reset_index()

            if i == 0 and j == 0:
                df = df[['date', 'index']]
            else:
                df = df[['index']]

            df = df.rename(columns={'index': target_region})

            res.append(df)
    # 最终形成城市+区域ID形式的迁移人流量大宽表
    df = pd.concat(res, axis=1)

    file_to_save = os.path.join(output_path, "region_migration.csv")
    print("saving result to %s" % file_to_save)
    # format: [date, A, B, C, D, E]
    df.to_csv(file_to_save, index=False, float_format = '%.2f')

In [ ]:
region_migration_process('./dataset/train_data', ["A", "B", "C", "D", "E"], [118, 30, 135, 75, 34], './dataset')

saving result to ./dataset/region_migration.csv


In [ ]:
pd.read_csv('dataset/region_migration.csv').head()

,date,A_0,A_1,A_2,A_3,A_4,A_5,A_6,A_7,A_8,...,E_24,E_25,E_26,E_27,E_28,E_29,E_30,E_31,E_32,E_33
0,21200501,171.82,200.06,231.88,209.15,403.13,179.94,204.45,897.98,1156.07,...,63.98,50.18,50.74,64.47,21.42,7.04,11.47,NaN,9.93,3.14
1,21200502,157.22,183.06,212.17,191.38,368.87,164.64,187.07,821.66,1057.82,...,56.44,44.27,44.76,56.87,18.90,6.21,10.12,NaN,8.76,2.77
2,21200503,204.28,237.86,275.68,248.66,479.28,213.93,243.07,1067.61,1374.46,...,70.30,55.13,55.75,70.83,23.54,7.73,12.60,NaN,10.91,3.45
3,21200504,163.38,190.24,220.49,198.89,383.34,171.10,194.41,853.88,1099.30,...,58.88,46.18,46.69,59.33,19.72,6.47,10.55,NaN,9.14,2.89
4,21200505,153.91,179.21,207.71,187.35,361.11,161.18,183.14,804.38,1035.57,...,57.64,45.20,45.71,58.07,19.30,6.34,10.33,NaN,8.95,2.83


In [ ]:
%run main.py --batch_size 1

[INFO] 2020-06-26 14:28:43,150 [     main.py:  219]:	Namespace(Ks=3, Kt=4, adj_mat_file='./dataset/adj_matrix.csv', batch_size=1, blocks=[[1, 16, 32], [32, 16, 64]], city_num=392, epochs=10, feat_dim=1, input_file='./dataset/region_migration.csv', keep_prob=1.0, label_file='./dataset/infection.csv', lr=0.005, n_his=20, n_pred=1, opt='ADAM', output_path='../outputs/', region_names_file='./dataset/region_names.txt', save=5, seed=1, submit_file='./dataset/train_data/submission.csv', test_num=1, use_cuda=False, val_num=3)
[INFO] 2020-06-26 14:28:44,674 [     main.py:   39]:	num examples: 26
[INFO] 2020-06-26 14:28:44,675 [     main.py:   45]:	Train examples: 22
[INFO] 2020-06-26 14:28:44,676 [     main.py:   46]:	Test examples: 1
[INFO] 2020-06-26 14:28:44,676 [     main.py:   50]:	Valid examples: 3


region migration:         date      A_0      A_1      A_2      A_3      A_4      A_5      A_6  \
0  21200501  0.17182  0.20006  0.23188  0.20915  0.40313  0.17994  0.20445   
1  21200502  0.15722  0.18306  0.21217  0.19138  0.36887  0.16464  0.18707   
2  21200503  0.20428  0.23786  0.27568  0.24866  0.47928  0.21393  0.24307   
3  21200504  0.16338  0.19024  0.22049  0.19889  0.38334  0.17110  0.19441   
4  21200505  0.15391  0.17921  0.20771  0.18735  0.36111  0.16118  0.18314   

       A_7      A_8   ...        E_24     E_25     E_26     E_27     E_28  \
0  0.89798  1.15607   ...     0.06398  0.05018  0.05074  0.06447  0.02142   
1  0.82166  1.05782   ...     0.05644  0.04427  0.04476  0.05687  0.01890   
2  1.06761  1.37446   ...     0.07030  0.05513  0.05575  0.07083  0.02354   
3  0.85388  1.09930   ...     0.05888  0.04618  0.04669  0.05933  0.01972   
4  0.80438  1.03557   ...     0.05764  0.04520  0.04571  0.05807  0.01930   

      E_29     E_30  E_31     E_32     E_33  
0  

[INFO] 2020-06-26 14:29:22,067 [     main.py:  103]:	epoch 1 | step 5 | loss 5141.372070
[INFO] 2020-06-26 14:29:57,194 [     main.py:  103]:	epoch 1 | step 10 | loss 14939.403320
[INFO] 2020-06-26 14:30:12,754 [     main.py:  114]:	valid result: | rmsle 0.3194172598309787 
[INFO] 2020-06-26 14:32:13,501 [     main.py:  103]:	epoch 1 | step 15 | loss 45625.109375
[INFO] 2020-06-26 14:32:48,288 [     main.py:  103]:	epoch 1 | step 20 | loss 2329.699219
[INFO] 2020-06-26 14:33:03,864 [     main.py:  114]:	valid result: | rmsle 0.3262588392553634 
[INFO] 2020-06-26 14:33:38,870 [     main.py:  103]:	epoch 2 | step 25 | loss 1218.109741
[INFO] 2020-06-26 14:34:14,048 [     main.py:  103]:	epoch 2 | step 30 | loss 3981.177734
[INFO] 2020-06-26 14:34:30,069 [     main.py:  114]:	valid result: | rmsle 0.31293732133227786 
[INFO] 2020-06-26 14:36:28,008 [     main.py:  103]:	epoch 2 | step 35 | loss 51102.667969
[INFO] 2020-06-26 14:37:02,589 [     main.py:  103]:	epoch 2 | step 40 | loss 4952

# 交通流量预测
## 数据集介绍
### 路段属性表
每条道路的每个通行方向由多条路段（link）构成，数据集中会提供每条link的唯一标识，长度，宽度，以及道路类型。

![file](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/1095279213540/1530604123635_UwFWal5fag.jpg)

In [ ]:
gy_link_info = pd.read_csv('data/data40468/gy_link_info.txt',sep=';')
gy_link_info.head()

,link_ID,length,width,link_class
0,4377906289869500514,57,3,1
1,4377906284594800514,247,9,1
2,4377906289425800514,194,3,1
3,4377906284525800514,839,3,1
4,4377906284422600514,55,12,1


###  link上下游关系表
link之间按照车辆允许通行的方向存在上下游关系，数据集中提供每条link的直接上游link和直接下游link。

![file](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/1095279213540/1530604200365_YWdoFt080j.jpg)

In [ ]:
gy_link_top = pd.read_csv('data/data40468/gy_link_top.txt',sep=';')
gy_link_top.head()

,link_ID,in_links,out_links
0,4377906289869500514,4377906285525800514,4377906281969500514
1,4377906284594800514,4377906284514600514,4377906285594800514
2,4377906289425800514,NaN,4377906284653600514
3,4377906284525800514,4377906281234600514,4377906280334600514
4,4377906284422600514,3377906289434510514#4377906287959500514,4377906283422600514


### link历史通行时间表
数据集中记录了历史每天不同时间段内（2min为一个时间段）每条link上的平均旅行时间，每个时间段的平均旅行时间是基于在该时间段内进入link的车辆在该link上的旅行时间产出。

![file](http://aliyuntianchipublic.cn-hangzhou.oss-pub.aliyun-inc.com/public/files/image/1095279213540/1530604254989_39m8JehJpp.jpg)

In [ ]:
!unzip data/data40468/travel_time_1.zip -d work/dataset/
!unzip data/data40468/travel_time_2.zip -d work/dataset/
!unzip data/data40468/travel_time_3.zip -d work/dataset/

Archive:  data/data40468/travel_time_1.zip
replace work/dataset/gy_link_travel_time_part1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  data/data40468/travel_time_2.zip
replace work/dataset/gy_link_travel_time_part2.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
df1 = pd.read_csv('work/dataset/gy_link_travel_time_part1.txt',sep=';')
df2 = pd.read_csv('work/dataset/gy_link_travel_time_part2.txt',sep=';', names=['link_ID','date','time_interval','travel_time'])
df2.drop(0,inplace=True)
df3 = pd.read_csv('work/dataset/gy_link_travel_time_part3.txt',sep=';')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
gy_link_travel_time = pd.concat([df1, df2, df3], axis=0)
gy_link_travel_time.tail()

,link_ID,date,time_interval,travel_time
10632039,4377906287663800514,2017-07-31,"[2017-07-31 14:16:00,2017-07-31 14:18:00)",76.9
10632040,4377906288663800514,2017-07-31,"[2017-07-31 07:08:00,2017-07-31 07:10:00)",3.4
10632041,4377906288663800514,2017-07-31,"[2017-07-31 17:12:00,2017-07-31 17:14:00)",5.1
10632042,4377906288663800514,2017-07-31,"[2017-07-31 17:38:00,2017-07-31 17:40:00)",36.6
10632043,3377906289044510514,2017-07-31,"[2017-07-31 17:08:00,2017-07-31 17:10:00)",11


In [ ]:
del df1
del df2
del df3
gc.collect()

In [ ]:
gy_link_travel_time['time_interval'] = gy_link_travel_time['time_interval'].str(1:20)

In [ ]:
len(gy_link_travel_time['link_ID'].unique())

264

In [ ]:
tmp = gy_link_travel_time[['time_interval', 'link_ID', 'travel_time']]

In [ ]:
tmp.to_csv('work/dataset/gy_link_travel_time.csv',index=None)

In [ ]:
tmp = pd.read_csv('work/dataset/gy_link_travel_time.csv',parse_dates=['travel_time'])

In [ ]:
tmp.head()

,time_interval,link_ID,travel_time
0,2016-05-21 23:20:00,9377906285566510514,17.6
1,2016-05-21 18:46:00,3377906288228510514,3.5
2,2016-05-21 07:06:00,3377906284395510514,10.0
3,2016-05-21 14:34:00,4377906284959500514,3.5
4,2016-05-21 05:04:00,9377906282776510514,1.5


In [ ]:
tmp['travel_time'] = tmp['travel_time'].astype(np.float16)

In [ ]:
tmp = tmp.groupby(['time_interval','link_ID']).agg({'travel_time': ['mean']})
tmp.columns = ['travel_time']
tmp.reset_index(inplace=True)

In [ ]:
tmp.head()

,time_interval,link_ID,travel_time
0,2016-03-01 00:00:00,3377906280028510514,4.601562
1,2016-03-01 00:00:00,3377906280395510514,22.500000
2,2016-03-01 00:00:00,3377906282328510514,20.000000
3,2016-03-01 00:00:00,3377906283328510514,6.601562
4,2016-03-01 00:00:00,3377906284028510514,19.593750


### 生成完整时间段序列，解决空值问题

In [ ]:
tmp['time_interval'].unique()

array(['2016-03-01 00:00:00', '2016-03-01 00:02:00',
       '2016-03-01 00:04:00', ..., '2017-07-31 17:54:00',
       '2017-07-31 17:56:00', '2017-07-31 17:58:00'], dtype=object)

In [ ]:
len(tmp['time_interval'].unique())

183624

In [ ]:
# 生成完整的时间段序列
ts = pd.Series(np.zeros(len(tmp['time_interval'].unique())), index=tmp['time_interval'].unique())
ts.to_csv("data/ts.csv", header=None)
ts = pd.read_csv("data/ts.csv", names=['time_interval', 'value'])

In [ ]:
# tmp = pd.merge(ts,tmp,how='left')
# tmp.drop(['value'], axis=1, inplace=True)

## 依样画葫芦
### 通行时间处理

In [ ]:
def car_process(tmp, ts, output_path):
    res = []
    link_name_list = []
        # order = sorted(range(region_nums[i]), key=lambda x:str(x))
    order = sorted(tmp['link_ID'].unique())
    for i, idx in enumerate(order):
        target_link = idx #str(idx)
        # pdb.set_trace()
        # 路段平均车速
        df = tmp[tmp['link_ID'] == target_link].reset_index(drop=True)
        df = pd.merge(ts,df,how='left')
        df.drop(['value'], axis=1, inplace=True)
        if i == 0:
            # 第一个路段要把时间给进去
            df = df[['time_interval', 'travel_time']]
        else:
            df = df[['travel_time']]

        df = df.rename(columns={'travel_time': '%d' % (idx)})
        link_name_list.append("%d" % (idx))

        res.append(df)
    df = pd.concat(res, axis=1)
    # 最终形成路段ID形式的平均车速大宽表
    file_to_save = os.path.join(output_path, "travel_time.csv")
    print("saving result to %s" % file_to_save)
    df.to_csv(file_to_save, index=False)

    link_name_file = os.path.join(output_path, "link_name_list.txt")
    with open(link_name_file, 'w') as f:
        names = ' '.join(link_name_list)
        # print(names)
        f.write(names)

In [ ]:
car_process(tmp,ts,'work/dataset')

saving result to work/dataset/travel_time.csv


In [ ]:
travel_time = pd.read_csv('work/dataset/travel_time.csv')

In [ ]:
travel_time.tail()

,time_interval,3377906280028510514,3377906280395510514,3377906281518510514,3377906281774510514,3377906282328510514,3377906282418510514,3377906283328510514,3377906284028510514,3377906284395510514,...,9377906282776510514,9377906283125510514,9377906283776510514,9377906284555510514,9377906285566510514,9377906285615510514,9377906286566510514,9377906286615510514,9377906288175510514,9377906289175510514
183619,2017-07-31 17:50:00,NaN,18.6,4.5,10.0,NaN,3.9,NaN,21.4,3.4,...,1.6,18.7,15.3,NaN,16.0,NaN,66.7,NaN,12.0,2.9
183620,2017-07-31 17:52:00,NaN,21.3,6.1,11.6,NaN,3.7,NaN,26.7,3.6,...,2.0,19.2,12.6,NaN,16.9,NaN,66.9,NaN,12.0,3.1
183621,2017-07-31 17:54:00,NaN,21.4,8.5,9.2,NaN,4.8,NaN,28.9,3.9,...,2.9,22.4,15.0,NaN,15.7,NaN,55.0,NaN,10.3,3.7
183622,2017-07-31 17:56:00,NaN,21.5,7.7,NaN,NaN,5.1,NaN,24.6,3.5,...,2.9,19.5,13.4,NaN,15.5,NaN,47.9,NaN,10.1,3.7
183623,2017-07-31 17:58:00,NaN,24.0,4.5,NaN,NaN,5.2,NaN,27.3,3.4,...,2.5,24.0,15.3,NaN,16.1,NaN,42.1,NaN,11.1,3.9


In [ ]:
travel_time['time_interval'].dropna(axis=0, how='any', inplace=True)

### 迁移车流量处理

In [ ]:
def process_car_migration(data_path, city_name):
    filename = os.path.join(data_path, "city_%s" % city_name, "migration.csv")
    migration = pd.read_csv(filename, 
                            sep=',', 
                            header=None,
                            names=['date', 's_city', 'e_city', city_name])

    # only use moving in "city" data, ignore moving out data
    df = migration[migration.e_city == city_name]
    df = df[["date", city_name]]

    # calculate total move in data of "city"
    df = df.groupby('date')[city_name].sum().reset_index()
    return df

# STGCN：时空图卷积网络
> 相关论文：[Spatio-Temporal Graph Convolutional Network \(STGCN\)](https://arxiv.org/pdf/1709.04875.pdf) 
在PGL中，提供了使用STGCN进行新冠疫情感染者趋势预测的示例。
## 数据集
需要将数据集按照下面的格式进行整理：
- input.csv: 历史迁移数据 with shape of [num\_time\_steps * num\_cities].

-  output.csv: 新增确诊数据 with shape of [num\_time\_steps * num\_cities].

- W.csv: 权重邻接矩阵 with shape of [num\_cities * num\_cities].

- city.csv: 城市列表.

### 开始训练

使用GPU训练示例
```
python main.py --use_cuda --input_file dataset/input_csv --label_file dataset/output.csv --adj_mat_file dataset/W.csv --city_file dataset/city.csv 
```

## 超参数

- n\_route: Number of city.
- n\_his: "n\_his" time steps of previous observations of historical immigration records.
- n\_pred: Next "n\_pred" time steps of New confirmed patients records.
- Ks: Number of GCN layers.
- Kt: Kernel size of temporal convolution.
- use\_cuda: Use gpu if assign use\_cuda. 

# STGCN: Spatio-Temporal Graph Convolutional Network

[Spatio-Temporal Graph Convolutional Network \(STGCN\)](https://arxiv.org/pdf/1709.04875.pdf) is a novel deep learning framework to tackle time series prediction problem. Based on PGL, we reproduce STGCN algorithms to predict new confirmed patients in some cities with the historical immigration records.

### Datasets

You can make your customized dataset by the following format:

* input.csv: Historical immigration records with shape of [num\_time\_steps * num\_cities].

* output.csv: New confirmed patients records with shape of [num\_time\_steps * num\_cities].

* W.csv: Weighted Adjacency Matrix with shape of [num\_cities * num\_cities].

* city.csv: Each line is a number and the corresponding city name.

### Dependencies

- paddlepaddle 1.6
- pgl 1.0.0

### How to run

For examples, use gpu to train STGCN on your dataset.
```
python main.py --use_cuda --input_file dataset/input_csv --label_file dataset/output.csv --adj_mat_file dataset/W.csv --city_file dataset/city.csv 
```

#### Hyperparameters

- n\_route: Number of city.
- n\_his: "n\_his" time steps of previous observations of historical immigration records.
- n\_pred: Next "n\_pred" time steps of New confirmed patients records.
- Ks: Number of GCN layers.
- Kt: Kernel size of temporal convolution.
- use\_cuda: Use gpu if assign use\_cuda. 


请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 